In [1]:
from ftx_client import FtxClient
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from supertrend import supertrend_analysis, get_supertrend_signals, calculate_ema, take_profit_calc
from config import API_KEY, API_SECRET
from pprint import pprint
import numpy as np
import time
from telegram_api_manager import TelegramAPIManager
import json
import os
import sys
import logging

plt.ioff()
FIGURE_PATH = "figures"

In [2]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    stream=sys.stdout, level=logging.INFO)
logger = logging.getLogger(__name__)

In [3]:
perpetuals = [
    "ETH-PERP", 
    "MATIC-PERP", 
    "SOL-PERP", 
    "DOGE-PERP", 
    "ADA-PERP", 
    "RUNE-PERP", 
    "CHZ-PERP", 
    "SUSHI-PERP",
    "DOT-PERP",
    "FTT-PERP",
    "STEP-PERP",
    "BTT-PERP",
    "FIL-PERP",
    "ICP-PERP"
]

In [ ]:
trades = {}
for perp in perpetuals:
    trades[perp] = []

tapi = TelegramAPIManager(group=False)

while(True):
    # Open new FTX Session
    ftx = FtxClient(api_key=API_KEY, api_secret=API_SECRET)

    for perp in perpetuals:
        df = ftx.get_historical_market_data(perp, interval="1h", start_time="10 days ago")

        # Perform supertrend analysis
        st, upt, dt = supertrend_analysis(df.high, df.low, df.close, look_back=10, multiplier=3)
        long_trigger, short_trigger, st_signal = get_supertrend_signals(df.close, st)

        # Test signal
        # st_signal = np.zeros(len(st_signal))
        # st_signal[-1] = -1
        
        # Check 200 EMA for confirmation
        ema200 = calculate_ema(df, time_period=200)
        
        # Set precision for orders
        precision = len(str(df.close[0]).split(".")[1])
        
        # Take profit calculator and stop loss
        stop_loss = round(st[-1], precision)
        long_profit_10pct, short_profit_10pct = take_profit_calc(df.close[-1], profit_percent=10, precision=precision)
        long_profit_5pct, short_profit_5pct = take_profit_calc(df.close[-1], profit_percent=5, precision=precision)
        
        # Plot and save results
        fig, ax = plt.subplots()
        ax.plot(df.index, df.close, ".-", color="tab:blue")
        ax.plot(df.index, ema200, color="tab:orange")
        ax.plot(df.index[1:], st, label=f"SL = {stop_loss} USD", color="tab:gray")
        
        ax.plot(df.index[1:], long_trigger, marker='^', color='tab:green', markersize=8, linewidth=0, label='Short')
        ax.plot(df.index[1:], short_trigger, marker='v', color='tab:red', markersize=8, linewidth=0, label='Long')
        ax.xaxis.set_major_formatter(mdates.DateFormatter('%b-%d %H:%m'))
        fig.autofmt_xdate()
        ax.set_title(perp)
        plt.close(fig)
        ax.legend()
        fig.savefig(os.path.join(FIGURE_PATH, f"{perp}.jpg"), dpi=300)

        # Check last element of signal array
        last_signal = st_signal[-1]
        # for idx, last_signal in enumerate(st_signal):
        if last_signal != 0:
            if last_signal == 1:
                # Long position
                new_position = {
                    "perp": perp, 
                    "position": "long", 
                    "entry": df.close[-1], 
                    "stop_loss": stop_loss, 
                    "10pctprofit": long_profit_10pct,
                    "5pctprofit": long_profit_5pct
                }
            elif last_signal == -1:
                # Short position
                new_position = {
                    "perp": perp, 
                    "position": "short", 
                    "entry": df.close[-1], 
                    "stop_loss": stop_loss, 
                    "10pctprofit": short_profit_10pct,
                    "5pctprofit": short_profit_5pct
                }

            if df.close[-1] < ema200[-1]:
                new_position["ema200"] = "under"
            else:
                new_position["ema200"] = "over"

            logger.info(new_position)
            trades[perp].append(new_position)
            tapi.send_photo(os.path.join(FIGURE_PATH, f"{perp}.jpg"), caption=json.dumps(new_position, indent=2, default=str))
        else:
            pass
    
    logger.info("Sleeping for 60 mins")
    ftx._session.close()
    time.sleep(60 * 60)

2021-07-13 21:53:15,495 - telegram_api_manager - INFO - cryptobot (@pymparoobot) is now running using Telegram personal chat id
2021-07-13 21:53:37,014 - __main__ - INFO - Sleeping for 60 mins
2021-07-13 22:53:58,304 - __main__ - INFO - Sleeping for 60 mins
2021-07-13 23:54:19,326 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 00:54:40,494 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 01:55:01,837 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 02:55:25,052 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 03:55:46,030 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 04:56:06,944 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 05:56:28,729 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 06:56:51,202 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 07:57:12,964 - __main__ - INFO - Sleeping for 60 mins
2021-07-14 08:57:34,146 - __main__ - INFO - Sleeping for 60 mins


In [ ]:
def calculate_pnl(perpetuals, trades, leverage=1):
    pnls = []
    for perp in perpetuals:
        if len(trades[perp]) >= 2:
            for idx in range(len(trades[perp])):
                if idx + 1 < len(trades[perp]):
                    if trades[perp][idx]["position"] == "short":
                        pnl = (trades[perp][idx]["entry"] - trades[perp][idx+1]["entry"]) / trades[perp][idx+1]["entry"] * 100 
                    elif trades[perp][idx]["position"] == "long":
                        pnl = (trades[perp][idx+1]["entry"] - trades[perp][idx]["entry"]) / trades[perp][idx]["entry"] * 100
                    pnl *= leverage
                    print(f"{perp} {round(pnl, 1)}%")
                    pnls.append(pnl)

    print(f"Average pnl = {sum(pnls) / len(pnls)}")

In [ ]:
calculate_pnl(perpetuals, trades, 10)

In [ ]:
print(json.dumps(new_position, indent=2, default=str))